In [241]:
import json
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, select, MetaData, Table
import requests
import sqlalchemy as sa
import urllib
import sql_scripts as ss
from datetime import date, datetime, timedelta
from threading import Thread

In [242]:
def getAuthforWMS(f_data):
    wmsAccess = f_data["wmsAccess"][0]
    return wmsAccess

In [243]:
#gets connections
def getConnforMYSQL(f_data, accessType):
    dialect = pyodbc.drivers()[-1]
    server = f_data[accessType][0]["server"]
    db = f_data[accessType][0]["database"]
    uid = f_data[accessType][0]["uid"]
    pwd = f_data[accessType][0]["pwd"]
    driver = f_data[accessType][0]["dialect_driver"]
    port = f_data[accessType][0]["port"]

    if accessType == "azureAccess":
        connection_string = (
            " Driver={%s}" %dialect +
            "; SERVER=%s" %server + 
            "; Database=%s " %db + 
            "; UID=%s" %uid +
            "; PWD=%s" %pwd
        )
        quoted = urllib.parse.quote_plus(connection_string)
        quoted = f_data[accessType][0]["dialect_driver"] + quoted
        engine = create_engine(quoted, fast_executemany=True).execution_options(isolation_level="AUTOCOMMIT")
    else:
        quoted = driver + uid + ":" + pwd + "@" + server + ":" + str(port) + "/" + db
        engine = create_engine(quoted).execution_options(isolation_level="AUTOCOMMIT")
        
    return engine

In [244]:
#Extracts data from DB
def extractionFunction(conn_integrator, sql_text):
    with conn_integrator.begin():
        str_sql = sa.text(sql_text)
        results = conn_integrator.execute(str_sql)
        columns = results.keys()

        list_columns = []
        list_rows = []
        #print column_names
        for column_name in columns:
            list_columns += [column_name]
        for row in results:
            list_rows += [row]

    return list_rows, list_columns

In [245]:
#get response from API
def setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers):
    #adds default headers
    headers['Accept'] =  "application/json"
    headers['Content-Type'] =  "application/json"   

    completeHTTP = schemeHTTP + baseHTTP + extraHTTP
    response = requests.get(completeHTTP, headers=headers)
    
    return response

In [246]:
class vtexOrder:
    #it's the class that represents a order, with or without PedidoID
    def __init__(self, orderID, pedidoID, jsonResponse, conn_azure):
        self.orderID = orderID
        self.pedidoID = pedidoID
        self.jsonResponse = jsonResponse
        self.conn_azure = conn_azure

        #cleans and translates the jsonResponse
        self.setValues()
        
        #[todo]
        #lista o nome dos discontos em self.jsonrespone['ratesAndBenefitsData']['rateAndBenefitsIdentifiers'] <- é uma lista, ver se o nome 'name' está na lista e fazer um string com o que tiver dentro
        
        if self.pedidoID is None:
            self.pedidoID = self.getNewPedidoID(conn_azure)
        
        sql_query = self.getUpsertAzureDBprocedure()
        print(sql_query)
        
    #gets ID_PEDIDO from the SEQ_ID_PEDIDO_VTEX and associates with the orderMainThread
    def getNewPedidoID(self, conn_azure):
        #rows_IdPedidoList, columns_IdPedidoList = extractionFunction(self.conn_azure, ss.queryList[self.file]['sql_querySequence'] )
        #return (rows_IdPedidoList[0])
        return 1
    
    #gets the jsonResponse and translate to variables also extracting the needed information
    #this was made this way for a better maintenance
    def setValues(self):
        self.ID_PEDIDO_VTEX       = self.pedidoID
        self.orderId              = self.orderID
        self.sequenceNumber       = self.jsonResponse['sequence']
        self.totalValue           = 0  if self.jsonResponse['value'] is None else self.jsonResponse['value']
        self.creationDate         = self.jsonResponse['creationDate']
        self.lastChange           = self.jsonResponse['lastChange']
        self.status               = self.jsonResponse['status']
        self.statusDescription    = self.jsonResponse['statusDescription']
        for item in self.jsonResponse['totals']:
            if item['id'] == 'Items':
                self.totalItems     = 0 if item['value'] is None else item['value']
            if item['id'] == 'Discounts':
                self.totalDiscounts = 0 if item['value'] is None else item['value']
            if item['id'] == 'Shipping':
                self.totalShipping  = 0 if item['value'] is None else item['value']
            if item['id'] == 'Tax':
                self.totalTax       = 0 if item['value'] is None else item['value']
        self.merchantName         = self.jsonResponse['merchantName']
        self.origin               = self.jsonResponse['origin']
        self.salesChannel         = self.jsonResponse['salesChannel']
        self.sellerOrderId        = self.jsonResponse['sellerOrderId']
        self.sellersId            = self.jsonResponse['sellers'][0]['id']
        self.sellersName          = self.jsonResponse['sellers'][0]['name']
        self.shippingListPrice    = 0        if self.jsonResponse['shippingData']['logisticsInfo'][0]['listPrice'] is None else self.jsonResponse['shippingData']['logisticsInfo'][0]['listPrice']


        self.selectedSla               = self.jsonResponse['shippingData']['logisticsInfo'][0]['selectedSla']
        self.deliveryCompany           = self.jsonResponse['shippingData']['logisticsInfo'][0]['deliveryCompany']
        self.shippingEstimate          = self.jsonResponse['shippingData']['logisticsInfo'][0]['shippingEstimate']
        self.shippingEstimateDate      = self.jsonResponse['shippingData']['logisticsInfo'][0]['shippingEstimateDate']
        self.accountHostname           = self.jsonResponse['hostname']
        self.isCompleted               = self.jsonResponse['isCompleted']
        self.isCheckedIn               = self.jsonResponse['isCheckedIn']
        self.marketplaceOrderId        = self.jsonResponse['marketplaceOrderId']
        self.marketplaceIsCertified    = self.jsonResponse['marketplace']['isCertified']
        self.marketplaceName           = self.jsonResponse['marketplace']['name']
        self.authorizedDate            = self.jsonResponse['authorizedDate']
        self.invoicedDate              = self.jsonResponse['invoicedDate']
        self.cancelReason              = self.jsonResponse['cancelReason']
        self.openTextField             = self.jsonResponse['openTextField']
        self.marketingDataUtmSource    = self.jsonResponse['marketingData']['utmSource']
        self.marketingDataUtmPartner   = self.jsonResponse['marketingData']['utmPartner']
        self.marketingDataUtmMedium    = self.jsonResponse['marketingData']['utmMedium']
        self.marketingDataUtmCampaign  = self.jsonResponse['marketingData']['utmCampaign']
        self.marketingDataCoupon       = self.jsonResponse['marketingData']['coupon']
        self.marketingDataUtmiCampaign = self.jsonResponse['marketingData']['utmiCampaign']
        self.marketingDataUtmipage     = self.jsonResponse['marketingData']['utmipage']
        self.marketingDataUtmiPart     = self.jsonResponse['marketingData']['utmiPart']
        self.discountsNames            = 'discountsNames'
        self.invoiceKey                = self.jsonResponse['packageAttachment']['packages'][0]['invoiceKey']

        self.userProfileId     = self.jsonResponse['clientProfileData']['userProfileId']
        self.email             = self.jsonResponse['clientProfileData']['email']
        self.firstName         = self.jsonResponse['clientProfileData']['firstName']
        self.lastName          = self.jsonResponse['clientProfileData']['lastName']
        self.documentType      = self.jsonResponse['clientProfileData']['documentType']
        self.document          = self.jsonResponse['clientProfileData']['document']
        self.phone             = self.jsonResponse['clientProfileData']['phone']
        self.corporateName     = self.jsonResponse['clientProfileData']['corporateName']
        self.tradeName         = self.jsonResponse['clientProfileData']['tradeName']
        self.corporateDocument = self.jsonResponse['clientProfileData']['corporateDocument']
        self.stateInscription  = self.jsonResponse['clientProfileData']['stateInscription']
        self.corporatePhone    = self.jsonResponse['clientProfileData']['corporatePhone']
        self.corporate         = self.jsonResponse['clientProfileData']['isCorporate']
        self.customerClass     = self.jsonResponse['clientProfileData']['customerClass']
        
        self.paymentId                    =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['id']
        self.paymentSystem                =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['paymentSystem']
        self.paymentSystemName            =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['paymentSystemName']
        self.paymentValue           = 0 if self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['value'] is None else self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['value']
        self.installments                 =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['installments']
        self.referenceValue               =aymentsReferenceValue:replaceEmpty(0)',
        self.paymentGroup                 =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['group']
        self.tid                          =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['tid']
        self.dueDate                      =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['dueDate']
        self.connectorResponsesTid        =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['connectorResponses']['Tid']
        self.connectorResponsesReturnCode =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['connectorResponses']['ReturnCode']
        self.connectorResponsesMessage    =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['connectorResponses']['Message']
        self.connectorResponsesNsu        =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['connectorResponses']['nsu']
        self.connectorResponsesAcquirer   =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['connectorResponses']['acquirer']
        self.billingAddressPostalCode     =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['postalCode']
        self.billingAddressCity           =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['city']
        self.billingAddressState          =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['state']
        self.billingAddressCountry        =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['country']
        self.billingAddressStreet         =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['street']
        self.billingAddressNumber         =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['number']
        self.billingAddressNeighborhood   =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['neighborhood']
        self.billingAddressComplement     =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['complement']
        self.billingAddressReference      =self.jsonResponse['paymentData']['transactions'][0]['payments'][0]['billingAddress']['reference']

    
    #the individual upsert into the AzureDB tables is done in each MainThread -> [OPT] maybe needs an upgrade later to be a batch insert if slow
    def getUpsertAzureDBprocedure(self):
        sql_query = f"""
        BEGIN TRY
            EXECUTE PROC_INSERT_PEDIDO_VTEX
                @ID_PEDIDO_VTEX= '{self.pedidoID}' ,
                @orderId='{self.orderID}',
                @sequenceNumber='{self.sequenceNumber}',
                @totalValue={self.totalValue},
                @creationDate='{self.creationDate}',
                @lastChange='{self.lastChange}',
                @status='{self.status}',
                @statusDescription='{self.statusDescription}',
                @totalItems={self.totalItems},
                @totalDiscounts={self.totalDiscounts},
                @totalShipping={self.totalShipping},
                @totalTax={self.totalTax},
                @merchantName='{self.merchantName}',
                @origin='{self.origin}',
                @salesChannel='{self.salesChannel}',
                @sellerOrderId='{self.sellerOrderId}',
                @sellersId='{self.sellersId}',
                @sellersName='{self.sellersName}',
                @shippingListPrice={self.shippingListPrice},

                @selectedSla='{self.selectedSla}',
                @deliveryCompany='{self.deliveryCompany}',
                @shippingEstimate='{self.shippingEstimate}',
                @shippingEstimateDate='{self.shippingEstimateDate}',
                @accountHostname='{self.accountHostname}',
                @isCompleted='{self.isCompleted}',
                @isCheckedIn='{self.isCheckedIn}',
                @marketplaceOrderId='{self.marketplaceOrderId}',
                @marketplaceIsCertified='{self.marketplaceIsCertified}',
                @marketplaceName='{self.marketplaceName}',
                @authorizedDate='{self.authorizedDate}',
                @invoicedDate='{self.invoicedDate}',
                @cancelReason='{self.cancelReason}',
                @openTextField='{self.jsonRespopenTextFieldonse}',
                @marketingDataUtmSource='{self.marketingDataUtmSource}',
                @marketingDataUtmPartner='{self.marketingDataUtmPartner}',
                @marketingDataUtmMedium='{self.marketingDataUtmMedium}',
                @marketingDataUtmCampaign='{self.marketingDataUtmCampaign}',
                @marketingDataCoupon='{self.marketingDataCoupon}',
                @marketingDataUtmiCampaign='{self.marketingDataUtmiCampaign}',
                @marketingDataUtmipage='{self.marketingDataUtmipage}',
                @marketingDataUtmiPart='{self.marketingDataUtmiPart}',
                @discountsNames ='{self.discountsNames}',
                @invoiceKey='{self.invoiceKey}',
        
                @userProfileId='{self.userProfileId}',
                @email='{self.email}',
                @firstName='{self.firstName}',
                @lastName='{self.lastName}',
                @documentType='{self.documentType}',
                @document='{self.document}',
                @phone='{self.phone}',
                @corporateName='{self.corporateName}',
                @tradeName='{self.tradeName}',
                @corporateDocument='{self.corporateDocument}',
                @stateInscription='{self.stateInscription}',
                @corporatePhone='{self.corporatePhone}',
                @corporate='{self.corporate}',
                @customerClass='{self.customerClass}',
                
                @paymentId='{self.paymentId}',
                @paymentSystem='{self.paymentSystem}',
                @paymentSystemName='{self.paymentSystemName}',
                @paymentValue={self.paymentValue},
                @installments='{self.installments}',
                @referenceValue= {self.referenceValue},
                @paymentGroup='{self.paymentGroup}',
                @tid='{self.tid}',
                @dueDate='{self.dueDate}',
                @connectorResponsesTid='{self.connectorResponsesTid}',
                @connectorResponsesReturnCode='{self.connectorResponsesReturnCode}',
                @connectorResponsesMessage='{self.connectorResponsesMessage}',
                @connectorResponsesNsu='{self.connectorResponsesNsu}',
                @connectorResponsesAcquirer='{self.connectorResponsesAcquirer}',
                @billingAddressPostalCode='{self.billingAddressPostalCode}',
                @billingAddressCity='{self.billingAddressCity}',
                @billingAddressState='{self.billingAddressState}',
                @billingAddressCountry='{self.billingAddressCountry}',
                @billingAddressStreet='{self.billingAddressStreet}',
                @billingAddressNumber='{self.billingAddressNumber}',
                @billingAddressNeighborhood='{self.billingAddressNeighborhood}',
                @billingAddressComplement='{self.billingAddressComplement}',
                @billingAddressReference='{self.billingAddressReference}'
        END TRY  
        BEGIN CATCH  
            SELECT   
                ERROR_NUMBER() AS ErrorNumber  
                ,ERROR_MESSAGE() AS ErrorMessage
                ,ERROR_LINE() AS ErrorLine
                ,ERROR_PROCEDURE() AS ErrorProcedure;  
        END CATCH 
        """
        return sql_query
    

In [247]:
class threadMain:
    #gets a index (a row=dictionary from a list), the columns and rows of the list, a connection to AzureDB, a file name to control which sql_query will be processed
    #and gets a global variables for the output lists - failed and succeded, the output must be global tables because of the multi threading
    #the mode is refering to get or not to get new idPedidos from azureDB
    def __init__(self, row, columns_List, conn_azure, file, outputSuccessTable, outputFailedTable):
        self.file = file
        self.columns_List = columns_List
        self.row = row
        self.conn_azure = conn_azure
        self.outputSuccessTable = outputSuccessTable
        self.outputFailedTable = outputFailedTable
        
        self.t = Thread(target=self.threadMainTask, args=( ))
        self.t.start()
    
    def getThread(self):
        return (self.t)
    
    #creates the threadMainTask for each row for the orders list
    def threadMainTask(self):
        #redeclare self variables
        columns_List = self.columns_List
        row = self.row
        outputSuccessTable = self.outputSuccessTable
        outputFailedTable = self.outputFailedTable

        #Define key variables for each order        
        for column_index in range(len(columns_List)):
            if columns_List[column_index] == "CONTA":
                account = row[column_index]
            if columns_List[column_index] == "CHAVE":
                key = row[column_index]
            if columns_List[column_index] == "TOKEN":
                token = row[column_index]
            if columns_List[column_index] == "orderId":
                orderID = row[column_index]
            if columns_List[column_index] == "ID_PEDIDO_VTEX":
                pedidoID = row[column_index]

        #Log the mainThread
        print(f'Starting Thread orderID: {orderID}')

        schemeHTTP = "https://"
        baseHTTP = "%s.%s.com.br" % (account, "vtexcommercestable")
        headers = {                
                'X-VTEX-API-AppKey': key,
                'X-VTEX-API-AppToken': token
                }
        extraHTTP = "/api/oms/pvt/orders/%s" %(orderID)

        #get first response
        response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers)
        print("response orderID: %s status_code: %s" %(orderID, response.status_code))

        #good Responses
        if response.status_code == 200:
            #creates the vtexOrder object -> is responsible for everything about the order (insert, update and get new PedidoID number)
            order = vtexOrder(orderID, pedidoID, response.json(), self.conn_azure)
            globals()[outputSuccessTable] = globals()[outputSuccessTable] + [order]
        #Bad Responses
        else:
            globals()[outputFailedTable].append({'account': account, 'status_code': response.status_code, 'time': datetime.now(), 'order' : orderID })
    

In [248]:
def main (file):
    #open auth file
    auth = open('auth.json')
    auth_load = json.load(auth)

    #getting conn for azure
    engine_azure = getConnforMYSQL(auth_load, "azureAccess")
    conn_azure = engine_azure.connect()

    #-----------------------------------------------------------------------------------------------------------------------------
    #start the List of success and failed orders
    OutputSuccessTable = ss.queryList[file]['OutputSuccess']
    globals()[OutputSuccessTable] = []
    OutputFailedTable = ss.queryList[file]['OutputFailed']
    globals()[OutputFailedTable] = []

    #-----------------------------------------------------------------------------------------------------------------------------
    ##Gets ALL orders from the temp_tables
    rows_List, columns_List = extractionFunction(conn_azure, ss.queryList[file]['sql_query'])

    #list of main threads
    threads = []
    
    for row in rows_List:
        t = threadMain(row, columns_List, file, conn_azure, OutputSuccessTable, OutputFailedTable)
        threads.append(t.getThread())
    
    for t in threads:
        t.join()

    #-----------------------------------------------------------------------------------------------------------------------------
    #After getting the pedidos_vtex and orders from vtex 
    #for each item, check PedidosID to update or insert the data
    #for item in globals()[OutputSuccessTable]:
    #    print (item.jsonResponse)



    #df_orders = pd.DataFrame.from_dict(globals()[ordersOutputSuccessTable])
    #df2_orders = pd.DataFrame.from_dict(globals()[ordersOutputFailedTable])
    
    #fillNan with empty string
    #df_orders = df_orders.fillna("")
    #df2_orders = df2_orders.fillna("")


    #df.to_sql(ss.queryList[file]['resultSuccessTable'], engine_azure, if_exists='replace', index=False)
    #if len(globals()["errorList"]) > 0:
    #    df2.to_sql(ss.queryList[file]['resultFailedTable'], engine_azure, if_exists='append', index=False)

    conn_azure.close()
    

In [249]:
if __name__ == "__main__":
    file = "getOrders_temptables"
    main(file)
    print('getOrders_temptables: done')

Starting Thread orderID: 1348160505463-01
Starting Thread orderID: 1348160560234-01
Starting Thread orderID: 1348160652138-01
Starting Thread orderID: 1348160652140-01
Starting Thread orderID: 1348160652142-01
Starting Thread orderID: 1348160652144-01
Starting Thread orderID: 1348160652146-01
Starting Thread orderID: 1348160652148-01
Starting Thread orderID: 1348160652150-01
Starting Thread orderID: 1348170503359-01
Starting Thread orderID: 1348170505465-01
Starting Thread orderID: 1348170560240-01
Starting Thread orderID: 1348170652152-01
Starting Thread orderID: 1348170652158-01
Starting Thread orderID: 1348170652160-01
Starting Thread orderID: 1348170652162-01
Starting Thread orderID: 1348170652164-01
Starting Thread orderID: 1348170652166-01
Starting Thread orderID: 1348170652168-01
Starting Thread orderID: 1348170652170-01
Starting Thread orderID: 1348170652172-01
Starting Thread orderID: 1348170652174-01
Starting Thread orderID: 1348170652176-01
Starting Thread orderID: 134817065

Exception in thread Thread-4176 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
Exception in thread Thread-4179 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-4191 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lucio.lee\AppData\Local\Temp\ipykernel_77484\1563109832.py", line 56, in threadMainTask
    self.run()
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    sel

response orderID: 1348170505465-01 status_code: 200
response orderID: 1348170652158-01 status_code: 200
response orderID: 1348180505467-01 status_code: 200
response orderID: 1348180560244-01 status_code: 200
response orderID: 1348170652178-01 status_code: 200
response orderID: 1348160652146-01 status_code: 200

        BEGIN TRY
            EXECUTE PROC_INSERT_PEDIDO_VTEX

                @ID_PEDIDO_VTEX= '1' ,
                @orderId='1348160652146-01',
                @sequenceNumber='652146',
                @totalValue=13944,
                @creationDate='2023-07-20T16:18:24.9618273+00:00',
                @lastChange='2023-07-26T17:10:06.0003930+00:00',
                @status='invoiced',

                @statusDescription='Faturado',
                @totalItems=20748,
                @totalDiscounts=-8304,
                @totalShipping=1500,
                @totalTax=0,
                @merchantName='None',
                @origin='Marketplace',
                @salesChannel=

Exception in thread Thread-4183 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-4184 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
Exception in thread Thread-4175 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-4196 (threadMainTask):
Traceback (most recent call last):
  File "c:\Users\lucio.lee\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-4187 (threadMainTask):
Traceback (most recent c

response orderID: 1348190545119-01 status_code: 200

        BEGIN TRY
            EXECUTE PROC_INSERT_PEDIDO_VTEX

                @ID_PEDIDO_VTEX= '1' ,
                @orderId='1348190545119-01',
                @sequenceNumber='545119',
                @totalValue=10380,
                @creationDate='2023-07-20T19:01:31.2188966+00:00',
                @lastChange='2023-07-26T19:18:43.2479672+00:00',
                @status='invoiced',

                @statusDescription='Faturado',
                @totalItems=10002,
                @totalDiscounts=-1000,
                @totalShipping=1378,
                @totalTax=0,
                @merchantName='None',
                @origin='Marketplace',
                @salesChannel='1',
                
                @sellerOrderId='00-1348190545119-01',
                @sellersId='1',
                @sellersName='Schneider',
                @shippingListPrice=1378,

                
        
                @selectedSla='Expressa',
 